In [1]:
# 参考 https://github.com/QwenLM/Qwen-Agent

import pprint
import urllib.parse
import json5
from qwen_agent.agents import Assistant
from qwen_agent.tools.base import BaseTool, register_tool
import os

# 步骤 1（可选）：添加一个名为 `my_image_gen` 的自定义工具。
@register_tool('my_image_gen')
class MyImageGen(BaseTool):
    # `description` 用于告诉智能体该工具的功能。
    description = 'AI 绘画（图像生成）服务，输入文本描述，返回基于文本信息绘制的图像 URL。'
    # `parameters` 告诉智能体该工具有哪些输入参数。
    parameters = [{
        'name': 'prompt',
        'type': 'string',
        'description': '期望的图像内容的详细描述',
        'required': True
    }]

    def call(self, params: str, **kwargs) -> str:
        # `params` 是由 LLM 智能体生成的参数。
        prompt = json5.loads(params)['prompt']
        prompt = urllib.parse.quote(prompt)
        return json5.dumps(
            {'image_url': f'https://image.pollinations.ai/prompt/{prompt}'},
            ensure_ascii=False)





In [2]:
# 步骤 2：配置您所使用的 LLM。
llm_cfg = {
    # 使用 DashScope 提供的模型服务：
    'model': 'qwen-max',
    'model_server': 'dashscope',
    # 'api_key': 'sk-882e296067b744289acf27e6e20f3ec0',
    'api_key': os.getenv('DASHSCOPE_API_KEY'),
    # 如果这里没有设置 'api_key'，它将读取 `DASHSCOPE_API_KEY` 环境变量。

    # 使用与 OpenAI API 兼容的模型服务，例如 vLLM 或 Ollama：
    # 'model': 'Qwen2-7B-Chat',
    # 'model_server': 'http://localhost:8000/v1',  # base_url，也称为 api_base
    # 'api_key': 'EMPTY',
    # （可选） LLM 的超参数：
    'generate_cfg': {
        'top_p': 0.8
    }
}


In [30]:
# 步骤 3：创建一个智能体。这里我们以 `Assistant` 智能体为例，它能够使用工具并读取文件。
# system_instruction = '''你是一个乐于助人的AI助手。
# 在收到用户的请求后，你应该：
# - 首先绘制一幅图像，得到图像的url，
# - 然后运行代码`request.get`以下载该图像的url，
# - 最后从给定的文档中选择一个图像操作进行图像处理。
# 用 `plt.show()` 展示图像。
# 你总是用中文回复用户。'''

system_instruction = '''你是一个乐于助人的AI助手。
在收到用户的请求后，你应该：
- 首先判断用户是要你做什么
- 如果要你绘制一幅图像，那么你先要调用AI绘图工具进行画图，接着将图像的url地址告诉用户，以便点击查看。然后执行代码`request.get`以下载该图像到本地并打开,并告诉用户下载地址，
- 如果是其他问题，则需要对文件进行分析再做回答

你总是用中文回复用户。'''
tools = ['my_image_gen', 'code_interpreter']  # `code_interpreter` 是框架自带的工具，用于执行代码。
#files = ['./examples/resource/doc.pdf']  # 给智能体一个 PDF 文件阅读。
files = ['./浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf']  # 给智能体一个 PDF 文件阅读。
bot = Assistant(llm=llm_cfg,
                system_message=system_instruction,
                function_list=tools,
                files=files)

In [33]:



# 步骤 4：作为聊天机器人运行智能体。
messages = []  # 这里储存聊天历史。
while True:
    # 例如，输入请求 "绘制一只狗并将其旋转 90 度"。
    query = input('用户请求: ')
    if query == '-1':
        break
    # 将用户请求添加到聊天历史。
    messages.append({'role': 'user', 'content': query})
    # response = []
#     # current_index = 0
# #   print(f'messages={messages}\n')
# #    for response in bot.run(messages=messages):  #会返回多次，每次循环，content都在增加内容。
# #        print(f'\n my response is :{response}\n')
#         current_response = response[0]['content'][current_index:]
#         current_index = len(response[0]['content']) # 每次循环，content都在增加字符，将response[0]['content']的输出移到新的位置开始。
# #        print(current_response, end='') # 打印当前的字符，拼接起来。   
#     # print(f'\n my latest response is :{response[0]['content']}\n')
# #    print(f'\n my response is :{response}\n')
    response= list(bot.run(messages=messages)) #将生成器转换为列表
    last_response= response[-1]  #取出最后生成的完整的response
    contents = [str(item['content']) for item in last_response if item.get('content')]#取出response的各个字典元素中所有content的值
    combined_content = ''.join(contents) #将这些值拼成完整的回答。
    print(combined_content)
     # 将机器人的回应添加到聊天历史。
    messages.extend(last_response)

  
    

用户请求:  请画一幅母子平安图


2025-04-26 16:38:37,675 - memory.py - 116 - INFO - {"keywords_zh": ["母子平安图", "画"], "keywords_en": ["safe mother and child picture", "draw"], "text": "请画一幅母子平安图"}
2025-04-26 16:38:37,676 - simple_doc_parser.py - 411 - INFO - Read parsed ./浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf from cache.
2025-04-26 16:38:37,677 - doc_parser.py - 108 - INFO - Start chunking ./浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf (浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf)...
2025-04-26 16:38:37,678 - doc_parser.py - 126 - INFO - Finished chunking ./浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf (浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf). Time spent: 0.0010046958923339844 seconds.
2025-04-26 16:38:37,679 - base_search.py - 56 - INFO - all tokens: 2937
2025-04-26 16:38:37,679 - base_search.py - 59 - INFO - use full ref
2025-04-26 16:38:46,905 - utils.py - 174 - INFO - Downloading ./浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf to workspace\tools\code_interpreter\浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf...
2025-04-26 16:38:46,908 - utils.py - 191 - INFO - Finished downloading ./浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf to w

{image_url: "https://image.pollinations.ai/prompt/%E6%AF%8D%E5%AD%90%E5%B9%B3%E5%AE%89%E5%9B%BE"}这是您请求的母子平安图，您可以点击链接查看: ![母子平安图](https://image.pollinations.ai/prompt/%E6%AF%8D%E5%AD%90%E5%B9%B3%E5%AE%89%E5%9B%BE)

现在我将该图片下载到本地。
error:

```
NameError: name 'shutil' is not defined

```看起来在下载图片时遇到了一个小问题，因为我忘记导入`shutil`库。让我立即修正这个问题并重新尝试下载。
execute_result:

```
'D:\\cursorprj\\lecture07\\CASE-Qwen-Agent-RAG\\workspace\\tools\\code_interpreter\\mother_and_child_safety.jpg'
```母子平安图已经成功下载到本地，您可以点击以下路径查看图片: 

`D:\cursorprj\lecture07\CASE-Qwen-Agent-RAG\workspace\tools\code_interpreter\mother_and_child_safety.jpg` 

如果您在访问该路径时遇到任何问题，请告诉我。


用户请求:  -1
